In [ ]:
import pandas as pd
import numpy as np
from utils import storage, plot, helpers, dates

from IPython.display import display, Markdown
from matplotlib import pyplot



In [ ]:
today = dates.last_day
one_day = dates.one_day
yesterday = today - one_day

display(Markdown('# Песочница по статистике по коронавирусу COVID-19'))
display(Markdown('---'))
with helpers.setlocale_ctx('ru_RU'):
    display(Markdown(f"DISCLAIMER: Данные в отчетах охватывают период с '_{dates.first_day.date().strftime('%d %B, %Y')}_' по '_{dates.last_day.date().strftime('%d %B, %Y')}_'. \
        За \"Сегодня\" принят последний доступный день в отчетах"))

In [ ]:
display(Markdown("## Топ десять стран по количеству заболевших на \"Сегодня\""))
display(Markdown("В таблице представлены ТОП-10 стран отсортированны по количеству заболевших на \"Сегодня\", а также с данными по количеству заболевших за последние 7 дней"))

storage.get_countries_report_by_column("Confirmed_Change", start_date=today - one_day * 6).fillna(0).sort_values(by=today, axis = 1).iloc[:,-10:]

In [ ]:
display(Markdown("## Динамика ТОП-5 стран по количеству заболевших на \"Сегодня\""))
display(Markdown("Здесь и ниже суффикс SMA означает среднее количество за N дней."))

countries_list = list(storage.get_countries_report_by_column("Confirmed_Change", start_date=today).fillna(0).sort_values(by=today, axis = 1).iloc[:,-5:].columns)

pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()

column_name = 'Confirmed_Change'

for country in countries_list:
    country_df = storage.get_country_report(country)
    plot.bar_with_sma_line(ax, country_df[column_name], 10, country, 0.1)
ax.legend()
ax.set_xlim(pd.to_datetime('15-03-2020', dayfirst=True))
ax.set_ylim(bottom=100)
pyplot.yscale("log")
pyplot.show()

In [ ]:
pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()

all_countries = storage.get_countries_report_by_column(column_name).fillna(0).apply(helpers.normalize)
mean = all_countries.mean(axis=1)
median = all_countries.quantile(.5, axis=1)

plot.bar_with_sma_line(ax, mean, label="Среднее")
plot.bar_with_sma_line(ax, median, label="Медианное")
ax.legend()

ax.set_xlim(pd.to_datetime('01-03-2020', dayfirst=True))
#ax.set_ylim(bottom=0, top = 1)
pyplot.title("Нормализованное количество заболевших в день")
pyplot.show()


In [ ]:
column_name = "Confirmed_Change"
start_date = pd.to_datetime('01-04-2020', dayfirst=True)

moscow_df = storage.get_region_report("Russia", "Москва")[start_date:]
regions_df = storage.get_regions_report_by_column("Russia", column_name, exclude=["Москва"], start_date=start_date)


In [ ]:
pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()

regions_df['Total'] = 0
regions_df['Total'] = regions_df.sum(axis = 1)

plot.bar_with_sma_line(ax, regions_df.Total, 3, 'Регионы')
plot.bar_with_sma_line(ax, moscow_df[column_name], 3, 'Москва')
plot.key_russian_dates(ax)

#ax.set_ylim(bottom=5000, top = 6500)
ax.legend(loc='upper left')
ax.set_title('Количество заболевших в день в Москве и остальной России')
ax.grid(axis='y', color='black', linestyle='dashed', alpha=0.4)
pyplot.show()

In [ ]:
pyplot.figure(figsize=(24,12))

ax = pyplot.subplot(111)
ax.xaxis_date()

column_name = 'Confirmed_Change'

for region_name in ["Санкт-Петербург", "Ростовская обл.", "Краснодарский край", "Хабаровский край"]:
    region_df = storage.get_region_report("Russia", region_name)[start_date:]

    plot.bar_with_sma_line(ax, region_df[column_name], 7, region_name+'-'+column_name)
plot.key_russian_dates(ax)

#ax.set_ylim(bottom=5000, top = 6500)
ax.legend(loc='upper left')
pyplot.show()

In [ ]:
regions = ["Москва", "Санкт-Петербург", "Ростовская обл.", "Краснодарский край"] #list(utils.get_country_regions("Russia"))
figure = pyplot.figure(figsize=(24,12))
plot.report(figure, storage.get_country_report("Russia")[60:], "Россия")

for region in regions:
    figure = pyplot.figure(figsize=(24,12))
    plot.report(figure, storage.get_region_report("Russia", region)[60:], region)

In [ ]:
pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()

country_name = "Russia"
column_name = "Confirmed_Change"
all_regions = storage.get_regions_report_by_column(country_name,column_name,start_date=pd.to_datetime("01-04-2020",dayfirst=True)).apply(helpers.normalize)

for region in all_regions.columns:
    ax.bar(all_regions.index, all_regions[region], alpha = 1/len(all_regions), color = "Blue")

mean = all_regions.mean(axis=1)
median = all_regions.median(axis=1)
ax.plot(mean.index,mean, color = "Red", label = "Mean")
ax.plot(median.index,median, color = "Yellow", label="Median")
ax.set_ylim(bottom=0, top = 1)
ax.grid(axis='y', color='black', linestyle='dashed', alpha=0.4)
ax.legend()
pyplot.title("Нормализованное количество заболевших в день")
pyplot.show()


In [ ]:
t1 = storage.get_region_report("Russia","Санкт-Петербург")['Confirmed_Change'].rolling(window=4).sum()
t2 = t1.shift(periods=4)
res =(t1/t2).apply(lambda x: np.nan_to_num(x, posinf=0, neginf=0))
pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()

plot.bar_with_sma_line(ax, res, 14)
ax.set_xlim(pd.to_datetime("01-04-2020",dayfirst=True))
ax.set_ylim(0.8,2)
pyplot.show()

In [ ]:
t1

In [ ]:
t2

In [ ]:
pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()
plot.bar_with_sma_line(ax ,(storage.get_region_report("Russia", "Москва")["Confirmed_Change"] / storage.get_country_report("Russia")['Confirmed_Change'])*100)
ax.set_xlim(pd.to_datetime("01-04-2020",dayfirst=True))
